Training LightGBM on the fraud dataset.
- SMOTE for Upsampling
- F1 score for metric
- using early stopping
- Hyperopt for hyperparameter optimization

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE, SVMSMOTE 
from imblearn.combine import SMOTETomek, SMOTEENN
import lightgbm as lgb
import sklearn
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sklearn.metrics import f1_score


/usr/local/miniconda3/envs/py36/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
x_train = pd.read_csv('./data/x_train.csv').values
y_train = pd.read_csv('./data/y_train.csv').values[:,0]
x_test = pd.read_csv('./data/x_test.csv').values
y_test = pd.read_csv('./data/y_test.csv').values[:,0]


In [3]:
x_train_resampled, y_train_resampled = SMOTE(random_state = 42).fit_resample(x_train, y_train)

print('#pos labels unsampled:', sum(y_train == 1))
print('#neg labels unsampled::', sum(y_train == 0))
print('#pos labels resampled:', sum(y_train_resampled == 1))
print('#neg labels resampled::', sum(y_train_resampled == 0))

#pos labels unsampled: 394
#neg labels unsampled:: 227451
#pos labels resampled: 227451
#neg labels resampled:: 227451


In [4]:
train_data = lgb.Dataset(x_train_resampled, label=y_train_resampled)
test_data = lgb.Dataset(x_test, label=y_test)


In [5]:
def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True


In [6]:

def objective(params):
    #print(params)
    
    evals_result = {}

    num_leaves = int(params['num_leaves'])
    min_data_in_leaf = int(params['min_data_in_leaf'])
    max_bin = int(params['max_bin'])
    bagging_fraction = params['bagging_fraction']
    bagging_freq = int(params['bagging_freq'])
    feature_fraction = params['feature_fraction']
    lambda_l2 = params['lambda_l2'],
    min_gain_to_split = params['min_gain_to_split']

    param = {'num_leaves':num_leaves, 
             'min_data_in_leaf':min_data_in_leaf, 
             'max_bin':max_bin,
             'learning_rate':0.1,
             'num_trees':1000, 
             'objective':'binary',
             'bagging_fraction':bagging_fraction,
             'bagging_freq':bagging_freq,
             'feature_fraction':feature_fraction,
             'verbose':-1,
             'lambda_l2':lambda_l2,
             'min_gain_to_split':min_gain_to_split,
             #'metric' : 'binary_logloss' # map, MAP, aliases: mean_average_precision
            }
    
    bst = lgb.train(param, 
                    train_data, 
                    valid_sets=[test_data], 
                    early_stopping_rounds=10, 
                    verbose_eval=False,
                    feval=lgb_f1_score, 
                    evals_result=evals_result,
                   )

    f1 = max(evals_result['valid_0']['f1'])
    
    return -f1


In [7]:
trials = Trials()

space = {
         'num_leaves' : hp.quniform('num_leaves', 100, 600, 10),
         'min_data_in_leaf' : hp.quniform('min_data_in_leaf', 10, 30, 1),
         'max_bin' : hp.quniform('max_bin', 200, 2000, 10),
         'bagging_fraction' : hp.uniform('bagging_fraction', 0.01, 1.0),
         'bagging_freq' : hp.quniform('bagging_freq', 0, 10, 1),
         'feature_fraction' :  hp.uniform('feature_fraction', 0.5, 1.0),
         'lambda_l2' : hp.uniform('lambda_l2', 0.0, 80.0),
         'min_gain_to_split' : hp.uniform('min_gain_to_split', 0.0, 1.0),
        }

best = fmin(objective,
    space=space,
    algo=tpe.suggest,
    trials=trials,
    max_evals=200)

print('#best', best)

print('#min(trials.losses())', min(trials.losses()))


  0%|          | 0/200 [00:00<?, ?it/s, best loss: ?]

/usr/local/miniconda3/envs/py36/lib/python3.6/site-packages/lightgbm/engine.py:116: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


 50%|████▉     | 99/200 [38:17<45:50, 27.23s/it, best loss: -0.8316831683168318]  


KeyboardInterrupt: 